# Manage notebooks & scripts

Call {meth}`~lamindb.core.Context.track` to track code along with its inputs and outputs.

```{note}

Tracking of notebooks & scripts is analogous to tracking data transformations through pipelines, functions & UI, see {doc}`docs:project-flow`.
```

In [ ]:
# !pip install 'lamindb[jupyter]'
!lamin init --storage ./test-track

## Initiate tracking

Call {meth}`~lamindb.core.Context.track` to register a data transformation and start tracking inputs & outputs of a run. You will find your notebook in the {class}`~lamindb.Transform` registry along with scripts, pipelines & functions. {class}`~lamindb.Run` stores executions.

In case `ln.context.uid` isn't yet set, `ln.context.track()` raises an exception and auto-generates a `uid` for you.

In [ ]:
import lamindb as ln

ln.context.uid = "9priar0hoE5u0000"  # <-- if undefined, auto-generated by ln.context.track()
ln.context.track()

LaminDB now automatically tracks all input and output data.

## Save run report, environment & source code

If you want to save the run report & source code for a notebook or script, call:

```python
ln.context.finish()
```

This is how a notebook with run report looks on the hub:

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8dJfH.png" width="900px">

## Query for a notebook or script

In the API, filter {class}`~lamindb.Transform` to obtain a transform record:

```python

transform = ln.Transform.get(name="Track notebooks & scripts")
transform.source_code  # source code
transform.latest_run.report  # report of latest run
transform.latest_run.environment  # environment of latest run
transform.runs  # all runs
```

On the hub, use the search or filter in the `Transform` view.

## Sync scripts with GitHub

To sync with your git commit, add the following line to your script:

```python
ln.settings.sync_git_repo = <YOUR-GIT-REPO-URL>
```

A tracked Python script could look like this:

```python
# my_script.py

import lamindb as ln

# set this setting in the script or on laminhub
ln.settings.sync_git_repo = "https://github.com/..."

# initiate tracking
ln.context.uid = "8a12ar0hoE5u0000"
ln.context.track()

# load input artifacts
data = ln.Artifact.get(...).load()

# save output artifacts
ln.Artifact(output_data, ...).save()

# save the script as transform.source_code
ln.context.finish()
```

You'll now see the GitHub emoji clickable on the hub:

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/IpV8Kiq4xUbgXhzlUYT7.png" width="900px">
<br>

## Manage templates

Save the template:

```bash
lamin save template1.ipynb --key templates/template1.ipynb --description "Template for analysis type 1" --registry artifact
```

Load the template:

```bash
lamin get https://lamin.ai/account/instance/artifact/Akd7gx7Y9oVOJlei0001
```

In [ ]:
# clean up test instance
!lamin delete --force test-track